## Preprocessing Playground
How to get a tf vector for each document 


In [61]:
import os

os.chdir("/Users/pascalweiss/dev/python/dl_project_gl_pw/src")

from src.preprocessing.preprocessing import Preprocessing
from src.preprocessing.cleaning import Cleaning
import pandas as pd
from gensim import corpora, models
import gensim
from spacy.attrs import ORTH, LEMMA
from collections import Counter

pd.options.display.max_colwidth = 0
pd.options.display.expand_frame_repr = False


### Loading the data

In [50]:
pp = Preprocessing()
pp.load_data('../data/comments_cleaned.txt', '../data/annotation.txt')
df = pp.get_data_df(deep_copy=False)[:1000]
cl = Cleaning(df)

### Frequency

In [91]:
foodata = [
    ["this", "is", "really", "really", "a", "document"],
    ["this", "is", "another"],
    ["completely", "new"]
]

In [102]:
def _flatten(list_of_lists):
    return [el for l in list_of_lists for el in l]


words = _flatten(foodata)
word_freq = Counter(words)
{x: word_freq[x] for x in word_freq if word_freq[x] >= 1}

In [95]:
common_words

### Preprocessing
Removing stop words, removing punctuation, etc.

In [4]:
rules = Cleaning.get_cleaning_rules(
    remove_stop=False,
    remove_punctuation=True,
    word_white_list=['!'],
    special_cases=[{'token': '&amp', 'replacement': [{ORTH: '&amp', LEMMA: 'and'}]}])

cl.apply_on_cols(cols=['post', 'reply'], cleaning_rules=rules, join_strings=False, n_threads=-1, batch_size=100)
df_c = cl.get_df()

### Splitting the data in test and train

In [6]:
pp.set_data_df(df_c, deep_copy=False)
pp.split_in_train_test()
df_train = pp.get_train_df(deep_copy=False)
df_test = pp.get_test_df(deep_copy=False)

In [11]:
dictionary = corpora.Dictionary(df_test.reply)
raw_corpus = [dictionary.doc2bow(d) for d in df_test.reply]

In [104]:
df_test

In [5]:
foodata = [
    ["this", "is", "really", "really", "a", "document"],
    ["this", "is", "another"],
    ["completely", "new"]
]
foodict = corpora.Dictionary(foodata)
foocorpus = [foodict.doc2bow(d) for d in foodata]

In [29]:
foocorpus

In [28]:
gensim.matutils.corpus2dense(foocorpus, num_terms=len(foodict)).T


### tf-idf

In [42]:
tfidf = models.TfidfModel(foocorpus)

In [48]:
gensim.matutils.corpus2dense(tfidf[foocorpus], num_terms=len(foodict)).T